In [1]:
import os
import re
import json
import pickle
import spacy
from typing import List, Set, Dict

# LangChain Imports
from langchain_core.documents import Document
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from mistralai import Mistral

# --- CONFIGURATION ---
# ⚠️ REPLACE WITH YOUR KEY (Do not hardcode in production)
MISTRAL_API_KEY = "iJ9SkFczJxTpP4Wy95onu0W4pGr38QH1"
MODEL_NAME = "open-mistral-7b"
DATA_DIR = "data"
CHARACTER_MAP_FILE = "story_characters.json"

# Initialize Clients
if not MISTRAL_API_KEY:
    print("⚠️ WARNING: Please set your MISTRAL_API_KEY.")
    client = None
else:
    client = Mistral(api_key=MISTRAL_API_KEY)

# Initialize SpaCy
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Downloading SpaCy...")
    from spacy.cli import download
    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

def clean_text(text):
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'\n\s*\n', '\n\n', text)
    return text.strip()

print("✅ Setup Complete.")

D:\Multi-Language Invoice Extractor\.venv1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Setup Complete.


In [2]:
def extract_characters_hybrid(text, story_title):
    """
    Extracts ALL characters from the full story text.
    Combines SpaCy (Speed) + Mistral (Intellect).
    """
    characters = set()

    # 1. SpaCy (Fast)
    doc = nlp(text[:100000])
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            characters.add(ent.text.strip())

    # 2. Mistral (Smart) - Only run on the first 4k chars to identify main cast
    if client:
        try:
            messages = [
                {"role": "user", "content": f"""
                Identify the distinct character names in this story.
                Return ONLY a JSON list of strings. Example: ["John", "Mary"].

                STORY: {story_title}
                TEXT: {text[:4000]}
                """}
            ]
            resp = client.chat.complete(
                model=MODEL_NAME,
                messages=messages,
                response_format={"type": "json_object"}
            )
            content = resp.choices[0].message.content
            llm_chars = json.loads(content)

            # Handle list or dict return
            if isinstance(llm_chars, list): characters.update(llm_chars)
            elif isinstance(llm_chars, dict):
                for val in llm_chars.values():
                    if isinstance(val, list): characters.update(val)
        except Exception as e:
            print(f"⚠️ Extraction skipped for {story_title}: {e}")

    # Clean duplicates and short noise
    return list({c for c in characters if len(c) > 2})

In [3]:
print("📂 Loading documents...")
dir_loader = DirectoryLoader(DATA_DIR, glob="**/*.txt", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})
raw_docs = dir_loader.load()

# Dictionary to store { "Story Title": ["Char1", "Char2"] }
global_character_map = {}
processed_docs = []

print(f"🕵️  Extracting characters for {len(raw_docs)} stories...")

for doc in raw_docs:
    full_text = doc.page_content
    lines = full_text.split("\n")
    # Heuristic: Assume first line is title, or filename
    title = lines[0].strip() if lines else "Unknown"
    body = clean_text(full_text)

    # Extract once per story
    story_chars = extract_characters_hybrid(body, title)
    global_character_map[title] = story_chars
    print(f"   -> {title}: Found {len(story_chars)} characters.")

    # Update doc metadata (Global context)
    doc.metadata["storyTitle"] = title
    doc.page_content = body
    processed_docs.append(doc)

# Save the Map separately
print(f"💾 Saving character map to {CHARACTER_MAP_FILE}...")
with open(CHARACTER_MAP_FILE, "w") as f:
    json.dump(global_character_map, f, indent=4)

📂 Loading documents...
🕵️  Extracting characters for 6 stories...
   -> A Mother: Found 26 characters.
   -> Sorrow: Found 7 characters.
   -> The Lantern Keepers: Found 6 characters.
   -> The Poor Relation’s Story: Found 14 characters.
   -> Mens Love: Found 22 characters.
   -> The Schoolmistress: Found 11 characters.
💾 Saving character map to story_characters.json...


In [5]:
def identify_chars_in_chunk(chunk_text: str, global_chars: List[str]) -> List[str]:
    """
    Checks which global characters appear in this specific chunk.
    This is pure Python (Fast), no LLM calls.
    """
    present = []
    chunk_lower = chunk_text.lower()
    for char in global_chars:
        # Check if name exists in chunk
        if char.lower() in chunk_lower:
            present.append(char)
    return present

print("✂️  Splitting and enriching chunks...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_chunks = text_splitter.split_documents(processed_docs)

# Enrich chunks with LOCAL character data
for chunk in final_chunks:
    title = chunk.metadata.get("storyTitle")
    # Get the master list for this story
    master_list = global_character_map.get(title, [])

    # Find which of these are in THIS specific chunk
    chunk_chars = identify_chars_in_chunk(chunk.page_content, master_list)

    # Store in metadata
    chunk.metadata["chars_in_chunk"] = chunk_chars

print(f"✅ Processed {len(final_chunks)} chunks. Example Metadata:")
print(final_chunks[0].metadata)

✂️  Splitting and enriching chunks...
✅ Processed 190 chunks. Example Metadata:
{'source': 'data\\a-mother.txt', 'storyTitle': 'A Mother', 'chars_in_chunk': ['Hoppy Holohan', 'Mrs Kearney', 'Holohan', 'Mr Holohan']}


In [6]:
print("🧠 Creating Vector Index...")
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db = FAISS.from_documents(final_chunks, embedding_model)
vector_db.save_local("faiss_index")

print("🔤 Creating BM25 Index...")
bm25_retriever = BM25Retriever.from_documents(final_chunks)
bm25_retriever.k = 5
with open("bm25_retriever.pkl", "wb") as f:
    pickle.dump(bm25_retriever, f)

print("✅ Indexing Complete!")

🧠 Creating Vector Index...
🔤 Creating BM25 Index...
✅ Indexing Complete!


In [8]:
def clean_and_validate_json(data):
    """
    Enforces schema but tries to fix common LLM mistakes before rejecting data.
    """
    if not isinstance(data, dict):
        return {"error": "Invalid JSON format"}

    structured_data = {
        "name": data.get("name", "Unknown"),
        "storyTitle": data.get("storyTitle", "Unknown"),
        "summary": data.get("summary", "Summary not available."),
        "relations": [],
        "characterType": data.get("characterType", "Unknown")
    }

    # Validate Relations with Fuzzy Key Matching
    raw_relations = data.get("relations", [])
    if isinstance(raw_relations, list):
        for item in raw_relations:
            if isinstance(item, dict) and "name" in item:
                # FIX: Check for 'relation' OR 'relationship'
                rel_value = item.get("relation") or item.get("relationship") or "Connected"

                structured_data["relations"].append({
                    "name": item["name"],
                    "relation": rel_value
                })

    return structured_data

def get_character_info(character_name, verbose=True):
    # 1. Load Map
    if os.path.exists(CHARACTER_MAP_FILE):
        with open(CHARACTER_MAP_FILE, "r") as f:
            global_map = json.load(f)
    else:
        return {"error": "Character map not found."}

    # 2. Smart Alias Detection
    character_exists = False
    detected_story = None
    canonical_name = character_name
    known_aliases = []

    for title, chars in global_map.items():
        matches = [c for c in chars if character_name.lower() in c.lower()]
        if matches:
            character_exists = True
            detected_story = title
            known_aliases = matches
            canonical_name = max(matches, key=len)
            break

    if not character_exists:
        return {"error": "Character not found."}

    # 3. Retrieve (Hybrid)
    try:
        vector_db = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)
        dense_retriever = vector_db.as_retriever(search_kwargs={"k": 6})
        with open("bm25_retriever.pkl", "rb") as f:
            sparse_retriever = pickle.load(f)
            sparse_retriever.k = 6

        dense_docs = dense_retriever.invoke(canonical_name)
        sparse_docs = sparse_retriever.invoke(canonical_name)
        if canonical_name != character_name:
             sparse_docs.extend(sparse_retriever.invoke(character_name))
    except Exception as e:
        dense_docs = []
        sparse_docs = []

    # Deduplicate & Filter
    all_docs = {d.page_content: d for d in sparse_docs + dense_docs}.values()
    filtered_docs = [d for d in all_docs if d.metadata.get("storyTitle") == detected_story]

    if not filtered_docs:
        filtered_docs = list(all_docs)

    # 4. Context Preparation
    context_text = ""
    for d in filtered_docs:
        local_chars = d.metadata.get("chars_in_chunk", [])
        context_text += f"""
        --- Snippet from "{d.metadata.get('storyTitle')}" ---
        [Characters in this scene: {local_chars}]
        Text: {d.page_content}
        """

    # 5. Generation (INFERENCE-ENCOURAGING PROMPT)
    messages = [
        {"role": "system", "content": "You are a JSON extractor. Output strictly valid JSON matching the provided schema."},
        {"role": "user", "content": f"""
        Task: Extract character details for "{canonical_name}" based ONLY on the text below.

        Aliases: The text may refer to them as {known_aliases} or with titles (Mr., Mrs.). Treat them as the same person.

        STORY TITLE: "{detected_story}"

        TEXT CONTEXT:
        {context_text}

        ### INSTRUCTIONS FOR RELATIONS ###
        - Extract clear relationships.
        - **IMPORTANT:** If no formal relationship (like "Father") is stated, **INFER** the relationship based on interactions.
          - Example: If they argue -> "Adversary" or "Conflict"
          - Example: If they work together -> "Colleague"
          - Example: If they talk -> "Acquaintance"

        ### REQUIRED OUTPUT FORMAT ###
        Example:
        {{
            "name": "Jon Snow",
            "storyTitle": "A Song of Ice and Fire",
            "summary": "Jon Snow is a brave leader...",
            "relations": [
                {{ "name": "Arya Stark", "relation": "Sister" }},
                {{ "name": "Cersei Lannister", "relation": "Enemy" }}
            ],
            "characterType": "Protagonist"
        }}

        ### YOUR OUTPUT FOR "{canonical_name}" ###
        """}
    ]

    try:
        resp = client.chat.complete(
            model=MODEL_NAME,
            messages=messages,
            response_format={"type": "json_object"}
        )
        # DEBUG: Print Raw JSON to see if LLM is generating keys we are deleting
        raw_json_str = resp.choices[0].message.content
        raw_json = json.loads(raw_json_str)

        if verbose:
            print(f"\n{'='*20} DEBUG: RAW LLM OUTPUT {'='*20}")
            print(json.dumps(raw_json, indent=2))
            print(f"{'='*60}\n")

        final_output = clean_and_validate_json(raw_json)
        final_output["name"] = canonical_name
        final_output["storyTitle"] = detected_story
        return final_output

    except Exception as e:
        return {"error": f"LLM Generation Failed: {e}"}

In [18]:
# --- TEST ---
# Set verbose=False to hide the debug prints and only show the final JSON
result = get_character_info("holohan", verbose=False)
print(json.dumps(result, indent=2))

{
  "name": "Hoppy Holohan",
  "storyTitle": "A Mother",
  "summary": "Hoppy Holohan, assistant secretary of the Eire Abu Society, is responsible for arranging concerts but is assisted by Mrs Kearney. He has a game leg and is known as Hoppy Holohan.",
  "relations": [
    {
      "name": "Mrs Kearney",
      "relation": "Colleague"
    },
    {
      "name": "Kathleen",
      "relation": "Acquaintance"
    },
    {
      "name": "Mrs Pat Campbell",
      "relation": "Acquaintance"
    },
    {
      "name": "O\u2019Madden Burke",
      "relation": "Acquaintance"
    },
    {
      "name": "Fitzpatrick",
      "relation": "Acquaintance"
    },
    {
      "name": "Meade",
      "relation": "Acquaintance"
    },
    {
      "name": "Madam Glynn",
      "relation": "Acquaintance"
    }
  ],
  "characterType": "Supporting"
}


In [14]:
!python --version



Python 3.11.9
